In [19]:
import os

import numpy as np
import torch
from pyro.contrib.examples.util import MNIST
import torch.nn as nn
import torchvision.transforms as transforms

import pyro
import pyro.distributions as dist
import pyro.contrib.examples.util  # patches torchvision
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import pickle

In [20]:
with open("data/sort-of-clevr.pickle", "rb") as fp:
    train_data, test_data = pickle.load(fp)

train_images = torch.tensor([x[0] for x in train_data]).float()
train_images = train_images.transpose(3, 1)
test_images = torch.tensor([x[0] for x in test_data]).float()
test_images = test_images.transpose(3, 1)

KeyboardInterrupt: 

In [3]:
def setup_data_loaders(batch_size=128, use_cuda=False):    
    kwargs = {'num_workers': 1, 'pin_memory': use_cuda}
    train_loader = torch.utils.data.DataLoader(dataset=train_images,
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(dataset=test_images,
        batch_size=batch_size, shuffle=False, **kwargs)
    return train_loader, test_loader

In [4]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, z_dim):
        super().__init__()
        self.input_dim = input_dim
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        self.fc22 = nn.Linear(hidden_dim, z_dim)
        
        self.softplus = nn.Softplus()
        
    def forward(self, x):
        x = x.reshape(-1, self.input_dim)
        hidden = self.softplus(self.fc1(x))
        mu = self.softplus(self.fc21(hidden))
        logsigma = self.softplus(self.fc22(hidden))
        
        return mu, logsigma

In [5]:
class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, z):
        hidden = self.softplus(self.fc1(z))
        out = self.sigmoid(self.fc2(hidden))
        
        return out

In [6]:
class VAE(nn.Module):
    
    def __init__(self, z_dim=64, hidden_dim=1024, data_dim=16875, use_cuda=False):
        super().__init__()
        self.encoder = Encoder(data_dim, hidden_dim, z_dim)
        self.decoder = Decoder(z_dim, hidden_dim, data_dim)
        
        if use_cuda:
            self.cuda()
            
        self.use_cuda = use_cuda
        self.z_dim = z_dim
    
    def model(self, x):
        pyro.module("decoder", self.decoder)
        with pyro.plate("data", x.shape[0]):
            z_loc = x.new_zeros(torch.Size((x.shape[0], self.z_dim)))
            z_scale = x.new_ones(torch.Size((x.shape[0], self.z_dim)))
            
            z = pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))
            loc_img = self.decoder(z)
            pyro.sample("obs", dist.Bernoulli(loc_img).to_event(1), obs=x.reshape(-1, 16875))
            
    def guide(self, x):
        pyro.module("encoder", self.encoder)
        with pyro.plate("data", x.shape[0]):
            z_loc, z_scale = self.encoder(x)
            pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))
        
    def reconstruct_img(self, x):
        z_loc, z_scale = self.encoder(x)
        z = dist.Normal(z_loc, z_scale).sample()
        loc_img = self.decoder(z)
        return loc_img

In [10]:
def train(svi, train_loader, use_cuda=False):
    epoch_loss = 0.
    for x in train_loader:
        if use_cuda:
            x = x.cuda()
        epoch_loss += svi.step(x)
        
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = epoch_loss / normalizer_train
    return total_epoch_loss_train

In [11]:
def evaluate(svi, test_loader, use_cuda=False):
    test_loss = 0.
    for x in test_loader:
        if use_cuda:
            x = x.cuda()
        test_loss += svi.evaluate_loss(x)
    normalizer_test = len(test_loader.dataset)
    total_epoch_loss_test = test_loss / normalizer_test
    return total_epoch_loss_test

In [22]:
LEARNING_RATE = 1.0e-4
USE_CUDA = False

NUM_EPOCHS = 500
TEST_FREQUENCY = 10

In [ ]:
train_loader, test_loader = setup_data_loaders(batch_size=256, use_cuda=USE_CUDA)

# clear param store
pyro.clear_param_store()

# setup the VAE
vae = VAE(use_cuda=USE_CUDA)

# setup the optimizer
adam_args = {"lr": LEARNING_RATE}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

train_elbo = []
test_elbo = []
# training loop
for epoch in range(NUM_EPOCHS):
    total_epoch_loss_train = train(svi, train_loader, use_cuda=USE_CUDA)
    train_elbo.append(-total_epoch_loss_train)
    print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))

    if epoch % TEST_FREQUENCY == 0:
        # report test diagnostics
        total_epoch_loss_test = evaluate(svi, test_loader, use_cuda=USE_CUDA)
        test_elbo.append(-total_epoch_loss_test)
        print("[epoch %03d] average test loss: %.4f" % (epoch, total_epoch_loss_test))
        with open(f"trained_vae_epoch_{epoch}.p", "wb") as fp:
            pickle.dump(vae, fp)

In [25]:
# train for another 100 epochs
for epoch in range(NUM_EPOCHS):
    total_epoch_loss_train = train(svi, train_loader, use_cuda=USE_CUDA)
    train_elbo.append(-total_epoch_loss_train)
    print("[epoch %03d]  average training loss: %.4f" % (epoch, total_epoch_loss_train))

    if epoch % TEST_FREQUENCY == 0:
        # report test diagnostics
        total_epoch_loss_test = evaluate(svi, test_loader, use_cuda=USE_CUDA)
        test_elbo.append(-total_epoch_loss_test)
        print("[epoch %03d] average test loss: %.4f" % (epoch, total_epoch_loss_test))

[epoch 000]  average training loss: 1424.3781
[epoch 000] average test loss: 1774.1999
[epoch 001]  average training loss: 1422.9729
[epoch 002]  average training loss: 1421.3735
[epoch 003]  average training loss: 1421.1764
[epoch 004]  average training loss: 1419.2823
[epoch 005]  average training loss: 1417.7131
[epoch 006]  average training loss: 1417.7184
[epoch 007]  average training loss: 1415.9502
[epoch 008]  average training loss: 1414.7416
[epoch 009]  average training loss: 1415.7711
[epoch 010]  average training loss: 1414.3245
[epoch 010] average test loss: 1782.6305
[epoch 011]  average training loss: 1411.7916
[epoch 012]  average training loss: 1409.0036
[epoch 013]  average training loss: 1409.7290
[epoch 014]  average training loss: 1408.7505
[epoch 015]  average training loss: 1406.4904
[epoch 016]  average training loss: 1405.5123
[epoch 017]  average training loss: 1404.4086
[epoch 018]  average training loss: 1405.6229
[epoch 019]  average training loss: 1401.528

KeyboardInterrupt: 

In [26]:
with open("trained_vae.p", "wb") as fp:
    pickle.dump(vae, fp)